# Preprocessing of the Small Database (6,922 edits)

This notebook documents the **preprocessing pipeline and methodological choices** used to clean and normalise the raw dataset obtained from the edit–article matching process.  
The objective of this step is to transform heterogeneous, noisy Wikipedia data into a **consistent and analytically reliable dataset**, suitable for downstream exploration, statistical analysis, and modelling.

The preprocessing relies on a single, reusable function applied uniformly to the entire dataset, ensuring **reproducibility and transparency**.

---

## Preprocessing Function Overview

The function below defines the complete cleaning and normalisation pipeline applied to the raw dataframe:

- temporal harmonisation,
- article name standardisation,
- weaponisation label normalisation,
- user-type inference,
- and column selection.

## Detailed Preprocessing Steps

### 1. Temporal Normalisation and Deduplication

- A `year` column is created from the edit timestamp to enable **yearly distribution analyses**.
- Duplicate edits are removed based on the `(article, date)` pair to avoid double counting.

This ensures temporal coherence and prevents artefacts due to duplicated records.


### 2. Article Title Cleaning and Normalisation

- Technical suffixes (e.g. `_analysis`) are removed.
- Underscores are replaced by spaces to match Wikipedia’s canonical naming conventions.
- Known article title changes are manually harmonised to ensure consistency across time.

This step is essential to avoid **artificial article fragmentation** in the analysis.


### 3. Weaponisation Label Harmonisation

- The `weaponised` field is normalised by:
  - lowercasing,
  - trimming whitespace,
  - and mapping heterogeneous labels to a controlled vocabulary:
    - `Weaponised`
    - `Not Weaponised`
    - `Unknown`
- Entries labelled as `Unknown` are excluded from further analysis to preserve interpretability.

This guarantees **semantic consistency** across analyses involving weaponisation.


### 4. User Type Inference

User types are inferred directly from the `user` field using heuristic rules:

- **Anonymous (IP)**  
  Detected via IPv4 / IPv6 regular expression matching.
- **Bot**  
  Detected via case-insensitive matching of the string `"bot"` in the username.
- **Registered**  
  Default category when neither anonymous nor bot conditions apply.

This classification allows systematic comparison of **editorial behaviour across user categories**.


### 5. Column Pruning and Dataset Simplification

- Intermediate columns used for inference (`is_anon`, `is_bot`) are removed.
- Non-essential or redundant columns inherited from the raw extraction are dropped.

The resulting dataframe retains only variables relevant to:
- editorial behaviour,
- user categorisation,
- temporal analysis,
- and weaponisation status.

---

## Output

The output of the preprocessing function is a **cleaned, standardised dataframe** that:
- is free of duplicates,
- uses consistent article and label naming,
- contains an explicit `user_type` variable,
- and is ready for exploratory analysis and modelling.


## Research Objective

This preprocessing step lays the foundation for all subsequent analyses.  
By enforcing strict data hygiene and explicit methodological choices, it ensures that any detected patterns — including **signals of weaponisation or coordinated behaviour** — are attributable to genuine editorial dynamics rather than artefacts of data collection or formatting.

The resulting dataset is designed to support feature extraction for the development of a downstream **Large Language Model (LLM)** focused on identifying and contextualising manipulative or strategic editing behaviours on Wikipedia.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import calendar
from tqdm import tqdm

In [2]:
df = pd.read_csv('../../datas/interim/Small Database Analysis/matched_edits_all.csv')

In [3]:
df

,article,user,date,comment,llm_output,weaponised,ngram,df_index,row_index_matched,fg_row_index,...,clean_before,clean_after,type_of_change_extracted,category_extracted_clean,propaganda_similarity,category_extracted_propaganda_mapped,aligned_before_chunk,aligned_after_chunk,similarity,significance_extracted
0,COVID-19_pandemic_in_Ukraine_analysis,Agathoclea,2020-03-11T20:56:06Z,removed [[Category:2019–20 coronavirus outbrea...,"Changed the category from ""2019–20 coronavirus...",Not Weaponised,annexation of Crimea by,0,15,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
1,History_of_Ukraine_analysis,Icey,2006-05-21T14:09:22Z,/* Further reading */ Disambiguation link repa...,Changed the reference format for Andrew Wilson...,Not Weaponised,A referendum in the,4,384,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
2,History_of_Ukraine_analysis,Irpen,2006-06-06T21:00:08Z,"this whole section doesn't belong here, speara...","Removed a section titled ""Ukraine and Nuclear ...",Not Weaponised,A referendum in the,4,389,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
3,History_of_Ukraine_analysis,193.60.161.100,2006-05-23T11:39:26Z,NaN,"Changed ""beyond"" to ""gayniss"" in the context o...",Not Weaponised,in the largely ethnic,4,383,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
4,History_of_Ukraine_analysis,Irpen,2006-06-14T17:49:44Z,revert to myself,Removed a POV (point of view) section regardin...,Not Weaponised,in the largely ethnic,4,392,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6917,COVID-19_pandemic_in_Ukraine_analysis,LSGH,2021-05-06T05:57:13Z,Updating number of cases in infobox,"Changed confirmed cases, recovery cases, death...",Not Weaponised,Donetsk and Luhansk oblasts,46,1180,798,...,"On 26 February 2014, Russian-speaking gunmen s...","--- \n+++ \n@@ -78,7 +78,7 @@\n \n \n \n-In la...",addition of phrase,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness","On 26 February 2014, Russian-speaking gunmen s...","\n \n During the Crimean crisis, demonstration...",0.562791,The addition of the phrase 'in what has been t...
6918,2014_pro-Russian_unrest_in_Ukraine_analysis,Garik 11,2014-04-08T17:05:13Z,/* Latvian citizen arrested */ more detail abo...,Changed the description of a Latvian citiz

In [26]:
def preprocess(dataframe):
    df = dataframe.copy()
    
    # Apply a year column in order to see a yearly distribution of edits
    df["year"] = pd.to_datetime(df["date"], errors="coerce").dt.year
    df.drop_duplicates(subset=['article', 'date'], inplace=True)
    
    # clean the article column name
    df['article'] = (
        df['article']
        .str.replace('_analysis', '', regex=False)
        .str.replace('_', ' ', regex=False)
    )
    
    # updates new wikipedia article name
    df["article"] = df["article"].replace(
        "Annexation of Crimea by the Russian Federation",
        "2014 Russian annexation of Crimea"
    )

    df["article"] = df["article"].replace(
        'Government of the Ukrainian People s Republic in exile',
        "Government of the Ukrainian People's Republic in exile"
    )
    
    # Normalizing the weaponised field because there are some capitalised and soem not --> lowercase and strip
    df["weaponised"] = df["weaponised"].astype(str).str.strip().str.lower()
    df["weaponised"] = df["weaponised"].replace({
        "true": "Weaponised",
        "false": "Not Weaponised",
        "unknown": "Unknown",
        "weaponised": "Weaponised",
        "not weaponised": "Not Weaponised"
    })
    
    df = df[df["weaponised"] != "Unknown"]
    
    # easy way to detected IP address
    ipv4_ipv6_pattern = (
        r"^(?:\d{1,3}\.){3}\d{1,3}$|"  # IPv4
        r"^(?:[A-Fa-f0-9]{0,4}:){2,7}[A-Fa-f0-9]{0,4}$"  # IPv6
    )

    # find what type of user a name is
    df["is_anon"] = df["user"].str.contains(ipv4_ipv6_pattern, na=False)
    df['is_bot'] = df['user'].str.contains('bot', case=False, na=False)
    
    def get_user_type(row):
        if row["is_anon"]:
            return "Anonymous (IP)"
        elif row["is_bot"]:
            return "Bot"
        else:
            return "Registered"        
    
    df['user_type'] = df.apply(get_user_type, axis=1)
    
    # clean all the useless columns
    def drop_useless_columns(dataframe):
        df = dataframe.copy()
        df = df.drop(['is_anon', 'is_bot'], axis = 1)
        columns = df.columns.to_list()
        df = df.drop([c for c in columns[6:22]], axis = 1)
        return df

    df = drop_useless_columns(df)

    return df

In [29]:
# apply the preprocessing function to the raw dataset
df_preprocess = preprocess(df)
df_preprocess

,article,user,date,comment,llm_output,weaponised,year,user_type
0,COVID-19 pandemic in Ukraine,Agathoclea,2020-03-11T20:56:06Z,removed [[Category:2019–20 coronavirus outbrea...,"Changed the category from ""2019–20 coronavirus...",Not Weaponised,2020,Registered
1,History of Ukraine,Icey,2006-05-21T14:09:22Z,/* Further reading */ Disambiguation link repa...,Changed the reference format for Andrew Wilson...,Not Weaponised,2006,Registered
2,History of Ukraine,Irpen,2006-06-06T21:00:08Z,"this whole section doesn't belong here, speara...","Removed a section titled ""Ukraine and Nuclear ...",Not Weaponised,2006,Registered
3,History of Ukraine,193.60.161.100,2006-05-23T11:39:26Z,NaN,"Changed ""beyond"" to ""gayniss"" in the context o...",Not Weaponised,2006,Anonymous (IP)
4,History of Ukraine,Irpen,2006-06-14T17:49:44Z,revert to myself,Removed a POV (point of view) section regardin...,Not Weaponised,2006,Registered
...,...,...,...,...,...,...,...,...
6917,COVID-19 pandemic in Ukraine,LSGH,2021-05-06T05:57:13Z,Updating number of cases in infobox,"Changed confirmed cases, recovery cases, death...",Not Weaponised,2021,Registered
6918,2014 pro-Russian unrest in Ukraine,Garik 11,2014-04-08T17:05:13Z,/* Latvian citizen arrested */ more detail abo...,Changed the description of a Latvian citizen b...,Not Weaponised,2014,Registered
6919,2014 pro-Russian unrest in Ukraine,Aleksandr Grigoryev,2014-04-19T15:24:13Z,/* Kidnapping of Ukrainian officials */ update,The change made in this revision is the additi...,Weaponised,2014,Registered
6920,Censuses in Ukraine,Aleksandr Grigoryev,2012-11-17T17:14:35Z,/* External links */ update,"Added a template for ""Ukraine topics"" and a ca...",Not Weaponised,2012,Registered


In [33]:
# store the preprocess database 
df_preprocess.to_csv('../../datas/final/small_db_preprocess.csv', index=False)